In [3]:
!pip install -U scikit-image

Requirement already up-to-date: scikit-image in /home/shervin/.local/lib/python3.8/site-packages (0.19.3)


In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2

In [12]:
def convolve(image, K):
    (iH,iW) = image.shape[:2]
    (kH,kW) = K.shape[:2]
    
    pad = (kW - 1) // 2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    output = np.zeros((iH,iW), dtype="float")
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            
            roi = image[y-pad:y+pad+1,x-pad:x+pad+1]
            k = (roi * K).sum()
            
            output[y-pad,x-pad] = k
    output = rescale_intensity(output, in_range=(0,255))
    output = (output * 255).astype('uint8')
    
    return output
    

In [4]:
smallBlur = np.ones((7,7), dtype="float")*(1.0/(7*7))
largeBlur = np.ones((21,21), dtype="float")*(1.0/(21*21))

In [5]:
sharpen = np.array((
    [0,-1,0],
    [-1,5,-1],
    [0,-1,0]
),dtype='int')

In [6]:
laplacian = np.array((
    [0,1,0],
    [1,-4,1],
    [0,1,0]
    ),dtype='int')

In [7]:
sobelX = np.array((
    [-1,0,1],
    [-2,0,2],
    [-1,0,1]
),dtype='int')

sobelY = np.array((
    [-1,-2,-1],
    [0,0,0],
    [1,2,1]
),dtype='int')

In [8]:
emboss = np.array((
    [-2,-1,0],
    [-1,1,1],
    [0,1,2]
),dtype='int')

In [9]:
kernelBank = (
    ("smallBlur",smallBlur),
    ('largeBlur',largeBlur),
    ('sharpen',sharpen),
    ('laplacian', laplacian),
    ('sobel_x',sobelX),
    ('sobel_y',sobelY),
    ('emboss',emboss)
)

In [13]:
image = cv2.imread('dog.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

for (kernelName, K) in kernelBank:
    
    print(f"[INFO] applying {kernelName} kernel")
    convolveOutput = convolve(gray, K)
    opencvOutput = cv2.filter2D(gray, -1, K)
    
    cv2.imshow("Original", gray)
    cv2.imshow(f'{kernelName} - convolve', convolveOutput)
    cv2.imshow(f'{kernelName} - opencv', opencvOutput)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


[INFO] applying smallBlur kernel
[INFO] applying largeBlur kernel
[INFO] applying sharpen kernel
[INFO] applying laplacian kernel
[INFO] applying sobel_x kernel
[INFO] applying sobel_y kernel
[INFO] applying emboss kernel
